#### Challenge 45: DSA parameter tampering

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp
import sha1

<div class="alert alert-block alert-info">

Take your DSA code from the previous exercise. Imagine it as part of an algorithm in which the client was allowed to propose domain parameters (the `p` and `q` moduli, and the `g` generator).

This would be bad, because attackers could trick victims into accepting bad parameters. Vaudenay gave two examples of bad generator parameters: generators that were `0 mod p`, and generators that were `1 mod p`.

Use the parameters from the previous exercise, but substitute `0` for `g`. Generate a signature. You will notice something bad. Verify the signature. Now verify any other signature, for any other string.
</div>

In [2]:
p = 0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
g = 0

# Generate a random public/private key pair:
x = random.randint(0, q-1)
y = pow(g, x, p)

msg = b'Test me'
[r, s, k] = cp.gen_DSA_sig(x, msg, p, q, g)
print(f'r={r}, s={s}, k={k}')
print(cp.check_DSA_sig(msg, y, r, s, p, g, q))

msg2 = b'BLAHBLAHBLAH'
print(cp.check_DSA_sig(msg2, y, r, s, p, g, q))



r=0, s=871088915603698057528183788525910709550489368534, k=521411376050990140685185409821947567417848461518
True
True


<div class="alert alert-block alert-info">

Now, try `(p+1)` as `g`. With this `g`, you can generate a magic signature `s`, `r` for any DSA public key that will validate against any string. For arbitrary `z`:

  $$ r = ((y^z) \mod p) \mod q $$

and
    
  $$ s = \frac{r}{z} \mod q $$

Sign "Hello, world". And "Goodbye, world".

</div>

In [3]:
bad_g = p+1
y = pow(bad_g, x, p)

z = random.randint(0, q-1)
evil_r = pow(y, z, p) % q
evil_s = (evil_r * cp.invmod(z, q)) % q

print(f'y={y}, r={evil_r}, s={evil_s}')

msg = b'Hello, world'
print(cp.check_DSA_sig(msg, y, evil_r, evil_s, p, bad_g, q))

msg = b'Goodbye, world'
print(cp.check_DSA_sig(msg, y, evil_r, evil_s, p, bad_g, q))

y=1, r=1, s=535610566950738117209897101874446532386215908437
True
True


Setting `g = p+1` forces `y=1` and `r=1`.  

Recall that:

4. Bob verifies the signature by computing:
    - `w = s^-1 mod q`
    - `u1 = (H(m) * w) mod q`
    - `u2 = (rw) mod q`
    - `v = ((g^u1 * y^u2) mod p) mod q`
5. If `v==r` the signature is verified

It also causes this to happen in the signature verication calculation:

`v = ((p+1)^u1 * 1^(u2) mod p`
`= 1 * 1`
`= 1`

The value of `m` and `s` don't matter.

!!


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)